In [1]:
# package(s) related to time, space and id
import datetime

# Used for mathematical functions
import math
import platform
import time

import matplotlib.pyplot as plt

# Used for making the graph to visualize our problem
import networkx as nx

# package(s) for data handling
import numpy as np
import pandas as pd

# spatial libraries
import pyproj
import shapely.geometry

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
from simplekml import Kml, Style

# OpenTNSim
import opentnsim

In [2]:
# V=ship velocity(VOG) or deep water velocity no current velocity was taking into account
# Urel=V+Ur
# Urel= 1/1-exp(4*(1-h/T))
# Am=midship cross section
# Ac=channel cross section
# T=ship draft

In [3]:
#Classification of water
h=1.5 #thelw na fortwsw pinaka na pairnei ta dedomena vathymetrias apo ola ta kanalia ths ollandias 
T=1.2 #thelw na valw lista me ola ta pithana draft
Am=37
Ac=200
if  h/T>3:# Deep water
    print ("not effect on ship")
    
elif h/T<1.5 and  h/T>1.2: #Shallow water
    print ("very significant effect on ships")
    
if  h/T<=1.2: # Very shallow water
    print ("water motion dominates ship")
     
# Froude number
def get_Frh (V,h):
        return V / np.sqrt(h*9.81)   
Frh = get_Frh (V=3,h=2) 
print('Frh', Frh)

# Frh<1 subcritical dn > dcritical
# Frh>1 supercritical dn < dcritical
def subcritical_or_supercritical (Frh):
    """Check flow"""
    if Frh<1:
        return "subcritical" 
    if  Frh<0.65: 
        return"desirable condition"
    if  Frh>1: # supercritical flow 
        return "unstable"
    print(subcritical_or_supercritical)
    return
Flow_type = subcritical_or_supercritical (Frh)
print (Flow_type)

# Return flow velocity
def get_Ur (V,h,T,Am,Ac):  
    return ((Am/Ac) * V)/(1-(Am/Ac)-(Frh**2))
Ur = get_Ur (V=3,h=1.5,T=1.2,Am=37,Ac=200)
#print('Ur=', Ur, 'm/s')

# Effective velocity
def get_Veff (V,h,T): 
    return V/(1-np.exp(4*(1- h/T)))
Veff = get_Veff (V=3,h=1.5,T=1.2)
#print('Veff=', Veff, 'm/s')

# Ratio Am/Ac 
def get_Urel (Am,Ac,V):
    if Am/Ac <=0.1:
        Urel= Veff
        return 
        else:
            Urel= V + Ur
            return
        return 
Urel = get_Urel (Am=1000,Ac=500,V=4.58)
print('Urel=', Urel, 'm/s')

SyntaxError: invalid syntax (1527817055.py, line 53)

In [ ]:
# Reynolds number computation
def RE(Ls):
    return (Urel*Ls)/ 10**(-6)
Re = RE (110)
print ('Re=',f'{Re:.1f}')

# Reynolds number
# Laminar flow Re <2000
# Unstable flow 2000< Re <4000
# Turbulent flow Re >4000
def lam_or_unstable_or_turbul(Re):
    """Check type of flow."""
    if Re<=2000:
        return "laminar"
    if Re>=4000: 
        return "turbulent"
    else: # unstable flow
        return "unstable"
    print(lam_or_unstable_or_turbul)
    return

Flow_condition = lam_or_unstable_or_turbul(Re)
Flow_condition

In [ ]:
#Frictional coefficient was computed based on ITTC theory & 40% of surcharge
def get_Cf (Re):
    Cf = 0.053/(((np.log(Re))-2)**2)
    return Cf
cf = get_Cf (Re)
print (f'{cf:.6f}')

In [ ]:
#Frictional resistance
def get_Rf (V,S):
    RF =  0.5 * 1000 *((V+Ur)**2) * cf * S
    return  RF
Rf = get_Rf (4.58, 400)
print ( 'Rf=',f'{Rf:.3f}' ,'kN')

In [ ]:
#Pressure resistance
def get_Rp (V,Am):
    RP = 0.5 * 1000 *((V+Ur)**2) * 0.15 * Am
    return RP
Rp = get_Rp (V=4.58, Am=37)
print ( 'Rp=',f'{Rp:.3f}','kN' )

In [ ]:
# Water level depression
def get_z (Am,Ac,h):
    Z = ((Am/Ac)*(Frh**2)*h)/(1-(Am/Ac)-(Frh**2))
    return Z
z = get_z (Am=37,Ac=500,h=3)
print ('z=',f'{z:.3f}','m')

In [ ]:
# Shallow water effect- Waterlevel depression
# Based on Schijf theory
def get_Rz (Am):
    z = get_z (Am=37,Ac=500,h=5)
    RZ = 1000 *9.81 *z * 0.2 * Am
    return RZ
Rz = get_Rz (Am=37.7)
print ('Rz=',f'{Rz:.3f}','kN')

In [ ]:
# Total resistance computation
def get_Rtot (Rf, Rp, Rz):
    R = Rf + Rp + Rz
    return R
Rtot = get_Rtot (Rf, Rp, Rz)
print ('Rtot=',f'{Rtot:.3f}','kN')

In [ ]:
# Power estimation 
# BHP > DHP > EHP
# 1) Effective horse power Pe (Power needed to overcome the total drag)
def get_Pe (V):
    PE = (V+Ur) * Rtot
    return PE
Pe = get_Pe (V = 4.58)
print ('Pe=',f'{Pe:.3f}','kw')

# 2) Delivered horse power Pd (Power delivered to the propeller shaft)
def get_Pd (Pe):
    PD = Pe/ 0.55
    return PD
Pd = get_Pd (Pe)
print ('Pd=',f'{Pd:.3f}','kw')

# 3) Brake horse power or Propulsion power Pb (Required motor power)
def get_Pb (Pe):
    PB = 2*Pe
    return PB
Pb = get_Pb (Pe)
print ('Pb=',f'{Pb:.3f}','kw')